In [1]:
from data_utils_ import *
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import logging
from collections import OrderedDict
import argparse
import numpy as np
from Models import Pipeline, MTLoss, Prediction #CrossAttention, Context
torch.cuda.empty_cache()
import gc
from types import SimpleNamespace

In [2]:
device = "cuda:7"

In [3]:
args = SimpleNamespace(dim = 768, upscale_dim = 256, codes='128,64,32,8', batch_size=2, learning_rate=0.001, weight_decay=0.07)

In [4]:
def getLoaders2(main_task_path = './Data/SignData/train_data', scaffold_task_path = './Data/2018/train_data', batch_size=8, slice=[-1, -1, -1], test_path='./Data/SignData/test_data'):
	print('Reading the Main Task Dataset...')
	main_task_dataset = RevSigData(main_task_path, mode='MAIN', slice_=slice[0], transform=Transform(), sigtx=ScaleSigScores())
	#main_task_dataset = dataset.readData(main_task_path, Transform(), mode='MAIN', n=slice[0])
	print('Reading the Scaffolds Task Dataset...')
	scaffold_task_dataset = RevSigData(scaffold_task_path, mode='SCAFFOLDS', slice_=slice[1], transform=Transform())
	#scaffold_task_dataset = dataset.readData(scaffold_task_path, Transform(), mode='SCAFFOLDS', n=slice[1])
	

	if test_path:
		print('Reading the test Dataset')
		test_dataset = RevSigData(test_path, mode='TEST', slice_=slice[2], transform=Transform(), sigtx=ScaleSigScores())
		#test_dataset = dataset.readData(test_path, Transform(), mode='TEST', n=slice[2])
	else:
		test_dataset = None


	#length of the both task datasets
	main_task_len = len(main_task_dataset)
	scaffold_task_len = len(scaffold_task_dataset)
	test_len = len(test_dataset)

	#inflate the smaller dataset to match the size of the larger one
	if main_task_len < scaffold_task_len:
		difference = scaffold_task_len - main_task_len
		sample = [random.choice(main_task_dataset) for _ in range(difference)]
		main_task_dataset = main_task_dataset + sample
	
	# print(len(main_task_dataset), len(scaffold_task_dataset))
	#print(main_task_len, scaffold_task_len)
	return (main_task_dataset, scaffold_task_dataset, test_dataset)

In [5]:
# main_task_dataset, scaffold_task_dataset, test_dataset = getLoaders2(batch_size=args.batch_size, slice=[-1,-1,-1])

In [6]:
# main_task_dataloader = DataLoader(main_task_dataset, batch_size = args.batch_size, shuffle = True, num_workers=4)
# scaffold_task_dataloader = DataLoader(scaffold_task_dataset, batch_size = args.batch_size, shuffle=True, num_workers=4)
# if test_dataset != None:
#     test_data_loader = DataLoader(test_dataset, batch_size=args.batch_size, shuffle=True, num_workers=4)
# else:
#     test_data_loader = None

In [7]:
# def evaluate(model, main_task_predictor, scaffold_task_predictor, Criterion, test_loader):
# 	with torch.no_grad():
# 		eval_loss = []
# 		for i, d in enumerate(test_loader,0):
# 			scaffold_task_data = d
# 			papers_sc, reviews_sc, recs_sc, confs_sc, sign_m = scaffold_task_data[0].transpose(1,2).float().to(device),\
# 							 scaffold_task_data[1].transpose(1,2).float().to(device), \
# 							 scaffold_task_data[2].float().to(device),\
# 							 scaffold_task_data[3].float().to(device),\
# 							 scaffold_task_data[4].float().to(device)

# 			ex, subj, opine = sign_m[:,0], sign_m[:,1], sign_m[:,2]
# 			out, rec_codes, conf_codes = model(papers_sc, reviews_sc)
# 			rec_preds, conf_preds = scaffold_task_predictor(rec_codes.view(out.shape[0], -1), conf_codes.view(out.shape[0], -1))

# 			#out_m, rec_codes_m, conf_codes_m = model(papers_sc, reviews_sc)
# 			ex_preds, subj_preds, intensity_preds = main_task_predictor(out, rec_codes, conf_codes)


# 			loss = Criterion([rec_preds.squeeze(1), conf_preds.squeeze(1), ex_preds.squeeze(1), subj_preds.squeeze(1), intensity_preds.squeeze(1)], [recs_sc, confs_sc, ex, subj, opine])
			
# 			eval_loss.append(loss.item())
# 		return np.average(eval_loss)



# def train(args, dataloaders):
#     main_task_loader, scaffold_task_loader, test_loader = dataloaders
#     model = Pipeline.Pipeline(args).to(device)
#     main_task_predictor = Prediction.MainPrediction(args.upscale_dim, args.upscale_dim, 16).to(device)
#     scaffold_task_predictor = Prediction.ScaffoldPrediction(args.upscale_dim, 8).to(device)

#     print(model)
#     for name, param in model.named_parameters():
#         print(name, param.shape)
#     print("No. of Trainable parameters {}".format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

#     Criterion = MTLoss.MTLoss().to(device2)
#     optimizer = torch.optim.Adam(list(model.parameters()) + list(Criterion.parameters()), lr=args.learning_rate, weight_decay=args.weight_decay) #+ list(main_task_predictor.parameters()) + list(scaffold_task_predictor.parameters())
#     optimizerMain = torch.optim.Adam(main_task_predictor.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
#     optimizerScaffold = torch.optim.Adam(scaffold_task_predictor.parameters(), lr=args.learning_rate, weight_decay=args.weight_decay)
#     epochs = 100

#     for epoch in range(epochs):
#         model.train()
#         epoch_loss = []
#         for i, d in enumerate(zip(scaffold_task_loader, main_task_loader),0):
#             #print(i)
#             main_task_data = d[1]
#             scaffold_task_data = d[0]
#             papers_sc, reviews_sc, recs_sc, confs_sc = scaffold_task_data[0].transpose(1,2).float().to(device),\
#                                  scaffold_task_data[1].transpose(1,2).float().to(device), \
#                                  scaffold_task_data[2].float().to(device),\
#                                  scaffold_task_data[3].float().to(device)

#             #print(ex.shape, subj.shape, opine.shape, recs_sc.shape, confs_sc.shape)

#             optimizer.zero_grad()
#             optimizerMain.zero_grad()
#             optimizerScaffold.zero_grad()
#             out, rec_codes, conf_codes = model(papers_sc, reviews_sc)
#             rec_preds, conf_preds = scaffold_task_predictor(rec_codes.view(out.shape[0], -1), conf_codes.view(out.shape[0], -1))
            
#             del papers_sc
#             del reviews_sc
            
#             papers_sc, reviews_sc, sign_m = main_task_data[0].transpose(1,2).float().to(device),\
#                                  main_task_data[1].transpose(1,2).float().to(device), \
#                                  main_task_data[2].float().to(device)

#             ex, subj, opine = sign_m[:,0], sign_m[:,1], sign_m[:,2]

#             #do the for the main task
#             out_m, rec_codes_m, conf_codes_m = model(papers_sc, reviews_sc)
#             ex_preds, subj_preds, intensity_preds = main_task_predictor(out_m, rec_codes_m, conf_codes_m)
#             #print(ex_preds.shape, subj_preds.shape, intensity_preds.shape)


#             loss = Criterion([rec_preds.squeeze(1), conf_preds.squeeze(1), ex_preds.squeeze(1), subj_preds.squeeze(1), intensity_preds.squeeze(1)], [recs_sc, confs_sc, ex, subj, opine])
#             epoch_loss.append(loss.item())
#             loss.backward()
#             optimizerMain.step()
#             optimizerScaffold.step()
#             optimizer.step()
#         #print("Epoch {} Loss: {:.3f}".format(epoch, np.average(epoch_loss)))
#             del papers_sc
#             del reviews_sc
#             gc.collect()
#         # 	break
#         # break

#         with torch.no_grad():
#             eval_loss = evaluate(model, main_task_predictor, scaffold_task_predictor, Criterion, test_loader)

#             print('Epoch: {} Train Loss: {:.6f}, Test Loss: {:.6f}'.format(epoch, np.average(epoch_loss),\
#                             eval_loss))
#             # print("Exhaustive {}".format(list(zip(ex_preds.data, ex.data))))
#             # print("Subjectivity {}".format(list(zip(subj_preds.data, subj.data))))
#             # print("Intensity {}".format(list(zip(intensity_preds.data, opine.data))))
#             # print("Recommendation {}".format(list(zip(rec_preds.data, recs_sc.data))))
#             # print("Confidence {}".format(list(zip(conf_preds.data, confs_sc.data))))

#             #logging.info('Predictions, Actual : {}'.format(str(list(zip(recs_preds_t, recs_sc_t)))))
#         #break


# def main(args, dataloaders=(main_task_dataloader, scaffold_task_dataloader, test_data_loader)):
#     train(args, dataloaders)




In [1]:
# main(args, dataloaders=(main_task_dataloader, scaffold_task_dataloader, test_data_loader))

###  Test the trained model

In [8]:
def evaluate(model, main_task_predictor, scaffold_task_predictor, test_loader, Criterion=None):
    with torch.no_grad():
        eval_loss = []
        ex_preds_, ex_actual = [], []
        subj_preds_, subj_actual = [], []
        int_preds_, int_actual = [], []
        rec_preds_, rec_actual = [], []
        conf_preds_, conf_actual = [], []
        for i, d in enumerate(test_loader,0):
            scaffold_task_data = d
            papers_sc, reviews_sc, recs_sc, confs_sc, sign_m = scaffold_task_data[0].transpose(1,2).float().to(device),\
                             scaffold_task_data[1].transpose(1,2).float().to(device), \
                             scaffold_task_data[2].float().to(device),\
                             scaffold_task_data[3].float().to(device),\
                             scaffold_task_data[4].float().to(device)

            ex, subj, opine = sign_m[:,0], sign_m[:,1], sign_m[:,2]
            out, rec_codes, conf_codes = model(papers_sc, reviews_sc)
            rec_preds, conf_preds = scaffold_task_predictor(rec_codes.view(out.shape[0], -1), conf_codes.view(out.shape[0], -1))

            #out_m, rec_codes_m, conf_codes_m = model(papers_sc, reviews_sc)
            ex_preds, subj_preds, intensity_preds = main_task_predictor(out, rec_codes, conf_codes)
            ex_preds_.append(ex_preds.item())
            ex_actual.append(ex.item())
            subj_preds_.append(subj_preds.item())
            subj_actual.append(subj.item())
            int_preds_.append(intensity_preds.item())
            int_actual.append(opine.item())



            if Criterion != None:
                loss = Criterion([rec_preds.squeeze(1), conf_preds.squeeze(1), ex_preds.squeeze(1), subj_preds.squeeze(1), intensity_preds.squeeze(1)], [recs_sc, confs_sc, ex, subj, opine])
                eval_loss.append(loss.item())
            else:
                eval_loss.append(0)

        return np.average(eval_loss), (ex_preds_, ex_actual), (subj_preds_, subj_actual), (int_preds_, int_actual)

In [10]:
device = 'cpu'
checkpoint = torch.load('./MODELS/exp3.pt', map_location=device)
args = SimpleNamespace(dim = checkpoint['dim'], upscale_dim = checkpoint['upscale_dim'], codes=checkpoint['codes']) 
model = Pipeline.Pipeline(args).to(device)
model.load_state_dict(checkpoint['model_state_dict'])
main_task_predictor = Prediction.MainPrediction(args.upscale_dim, args.upscale_dim, 32).to(device)
main_task_predictor.load_state_dict(checkpoint['main_state_dict'])
scaffold_task_predictor = Prediction.ScaffoldPrediction(args.upscale_dim, 8).to(device)
scaffold_task_predictor.load_state_dict(checkpoint['scaffold_state_dict'])
Criterion = MTLoss.MTLoss().to(device)
Criterion.load_state_dict(checkpoint['criterion_state_dict'])

<All keys matched successfully>

In [11]:
_, _, test_dataset = getLoaders2(batch_size=1, slice=[-1,-1,-1])
test_data_loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=4)

Reading the Main Task Dataset...
Reading the Scaffolds Task Dataset...
Reading the test Dataset


In [12]:
loss, exhaustive, subjective, opinion = evaluate(model, main_task_predictor, scaffold_task_predictor, test_data_loader, Criterion)

In [15]:
# list(zip(*opinion))

In [13]:
opine_pred, opine_actual = opinion
subj_pred, subj_actual = subjective
exhaustive_pred, exhaustive_actual = exhaustive


In [29]:
from numpy import dot
from numpy.linalg import norm
def cosine(a, b):
    return dot(a,b)/(norm(a)*norm(b))
def rmse(predictions, targets):
    return np.sqrt(np.mean((np.asarray(predictions)-np.asarray(targets))**2))

In [30]:
def get_metric(score):
    RMSE = rmse(score[0], score[1])
    sim = cosine(score[0], score[1])
    print("RMSE: {}, cosine similarity: {}".format(RMSE, sim))

In [33]:
for score in [(exhaustive_pred, exhaustive_actual), (subj_pred, subj_actual), (opine_pred, opine_actual)]:
    get_metric(score)

RMSE: 0.5017433881250435, cosine similarity: 0.9895367837062644
RMSE: 0.8054140323941106, cosine similarity: 0.982373976678173
RMSE: 1.45750686097663, cosine similarity: 0.9397649115706209


### Mean Baseline

In [34]:
exhaustive_pred = [np.average(exhaustive_actual)]*len(exhaustive_actual)
subj_pred = [np.average(subj_actual)]*len(exhaustive_actual)
opine_pred = [np.average(opine_actual)]*len(exhaustive_actual)
for score in [(exhaustive_pred, exhaustive_actual), (subj_pred, subj_actual), (opine_pred, opine_actual)]:
    get_metric(score)

RMSE: 1.5518540202548305, cosine similarity: 0.8932938675323975
RMSE: 2.1199669478437224, cosine similarity: 0.8705800832137459
RMSE: 1.6752461775372902, cosine similarity: 0.9184800659455056


### Examples

In [47]:
import os
import json
path = './Data/SignData/test/reviews'
reviews = os.listdir(path)
filenames = os.listdir('./Data/SignData/test_data/')

review_text = []
indices = []
for i, filename in enumerate(filenames):
    split = filename.split('_')
    fname, count = ''.join(c for c in split[:-1]), split[-1]
    if fname in reviews:
        rev = json.load(open(os.path.join(path, fname)))['reviews'][int(count)]
        review_text.append(rev)
    else:
        indices.append(i)

In [48]:
from collections import defaultdict
dict_ = defaultdict(lambda: {})
for i in range(len(opine_pred)):
    scores = {}
    if i not in indices:
        scores['layer1'] = (exhaustive_pred[i], exhaustive_actual[i])
        scores['layer2'] = (subj_pred[i], subj_actual[i])
        scores['layer3'] = (opine_pred[i], opine_actual[i])
        dict_[i] = scores
        

In [85]:
91, 89, 66, 62, 59, 54, (37)

(91, 89, 66, 62, 59, 54, 37)

In [49]:
import numpy as np
l1, l2 = np.array([0.05428571,0.93013972,-0.99966815]), np.array([33.26288336,252.25067107,0.99966679])
def normalize(s):
    s = np.array(s)
    sent = s[2]
    s = (s - l1)/(l2 - l1)
    s = (s*9) + 1
    s[2] = (((sent - (-1))*9)/2) + 1
    return s

In [35]:
# for i in [79]:
#     new_scores = normalize(review_text[i]['SCORES'])
#     print(new_scores)
#     print(review_text[i])

In [50]:
for i, review in enumerate(review_text):
    new_scores = normalize(review_text[i]['SCORES'])
    for k,v in dict_.items():
        if np.round(dict_[k]['layer1'][1], decimals=3) == np.round(new_scores[0], decimals=3):
            review_text[i]['PREDICTIONS'] = dict_[k]
            review_text[i]['NORMALISED'] = new_scores

In [36]:
# predictions = []
# for k,v in dict_.items():
#     pred = []
#     pred.append(np.round(dict_[k]['layer1'][0], 3))
#     pred.append(np.round(dict_[k]['layer2'][0], 3))
#     pred.append(np.round(dict_[k]['layer3'][0], 3))
#     predictions.append(pred)

In [37]:
# len(predictions)

In [31]:
tst_reviews

,RECOMMENDATION,REVIEW TITLE,comments,VARIANCE,CONFIDENCE,SCORES,ID,Layer 1,Layer 1 N,Layer 2,Layer 2 N,Layer 3,Layer 3 N
0,7,,"""Summary: \nThe authors present a simple varia...",0,4,"[16.587, 142.202, 0.044]",Hyp3i2xRb.json,16.587,5.481,142.202,6.059,0.044,5.699
1,4,,"""The paper entitled 'Siamese Survival Analysis...",0,5,"[1.28, 9.568, -0.34]",HkjL6MiTb.json,1.280,1.332,9.568,1.309,-0.340,3.968
2,3,,"""In this paper, the authors define a simulated...",0,3,"[11.172, 160.306, -0.672]",B1EGg7ZCb.json,11.172,4.013,160.306,6.707,-0.672,2.478
3,7,,"""Summary:\n The paper presents an unsupervised...",0,3,"[28.766, 328.435, -0.933]",HyI6s40a-.json,28.766,8.781,328.435,12.728,-0.933,1.299
4,7,,"""The paper introduces a new memory mechanism s...",0,4,"[3.718, 42.942, -0.455]",Bk9zbyZCZ.json,3.718,1.993,42.942,2.504,-0.455,3.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,7,,"""The paper presents a novel representation of ...",0,3,"[2.031, 21.92, 0.189]",HkOhuyA6-.json,2.031,1.536,21.920,1.752,0.189,6.349
77,6,,"""This paper investigates the complexity of neu...",0,5,"[4.023, 50.659, 0.685]",Sy-tszZRZ.json,4.023,2.076,50.659,2.781,0.685,8.583
78,6,,"""This paper is an extension of the \u201cproto...",0,5,"[6.816, 64.86, -0.322]",HJcSzz-CZ.json,6.816,2.832,64.860,3.289,-0.322,4.051
79,4,,"""The authors tackle the problem of estimating ...",0,4,"[6.889, 85.239, -0.781]",HkjL6MiTb.json,6.889,2.852,85.239,4.019,-0.781,1.985


In [38]:
# import pandas as pd
# tst_reviews = pd.read_csv('Test_Reviews.csv')
# tst_reviews["Predictions"] = predictions

In [1]:
# for i, rev in enumerate(review_text):
#     print(i)
#     print(rev['PREDICTIONS'])
#     print(rev['NORMALISED'])

In [23]:
# tst_indices = [0, 20, 50, 51] #, 43, 29, 28, 8, 3]
# for ind in tst_indices:
#     print(review_text[ind])
#     print('\n')


#### Study Reviews

In [54]:
# def Round(num):
#     return np.round(num, decimals=3)

# reviews = []
# import csv
# for i, review in enumerate(review_text):
#     print(i)
#     new_scores = list(map(lambda x: Round(x), normalize(review_text[i]['SCORES'])))
#     review['SCORES'] = list(map(lambda x: Round(x), review['SCORES']))
#     preds = [review_text[i]['PREDICTIONS']['layer1'][0], review_text[i]['PREDICTIONS']['layer2'][0], review_text[i]['PREDICTIONS']['layer3'][0]]
#     review['PREDICTIONS'] = list(map(lambda x: Round(x), preds))
#     review['Layer 1'], review['Layer 1 N'] = review['SCORES'][0], new_scores[0]
#     review['Layer 2'], review['Layer 2 N'] = review['SCORES'][1], new_scores[1]
#     review['Layer 3'], review['Layer 3 N'] = review['SCORES'][2], new_scores[2]
#     reviews.append(review)

In [52]:
field_names = reviews[0].keys()
with open('Test_Reviews_.csv', 'w') as csvfile: 
    writer = csv.DictWriter(csvfile, fieldnames = field_names) 
    writer.writeheader() 
    writer.writerows(reviews)

In [56]:
tst = pd.read_csv('Test_Reviews_.csv')
tst

,RECOMMENDATION,REVIEW TITLE,comments,VARIANCE,CONFIDENCE,SCORES,PREDICTIONS,NORMALISED,Layer 1,Layer 1 N,Layer 2,Layer 2 N,Layer 3,Layer 3 N
0,7,,"""Summary: \nThe authors present a simple varia...",0,4,"[16.587, 142.202, 0.044]","[5.478, 6.908, 3.191]",[5.48062988 6.05905833 5.69919155],16.587,5.481,142.202,6.059,0.044,5.699
1,4,,"""The paper entitled 'Siamese Survival Analysis...",0,5,"[1.28, 9.568, -0.34]","[1.449, 1.612, 4.07]",[1.33218592 1.30933134 3.96794648],1.280,1.332,9.568,1.309,-0.340,3.968
2,3,,"""In this paper, the authors define a simulated...",0,3,"[11.172, 160.306, -0.672]","[4.463, 5.594, 2.866]",[4.01313185 6.70739261 2.47807651],11.172,4.013,160.306,6.707,-0.672,2.478
3,7,,"""Summary:\n The paper presents an unsupervised...",0,3,"[28.766, 328.435, -0.933]","[8.405, 10.596, 2.652]",[ 8.78125838 12.72822394 1.29945041],28.766,8.781,328.435,12.728,-0.933,1.299
4,7,,"""The paper introduces a new memory mechanism s...",0,4,"[3.718, 42.942, -0.455]","[2.159, 2.73, 4.341]",[1.99291843 2.50449618 3.45302132],3.718,1.993,42.942,2.504,-0.455,3.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,4,,"""The majority of the paper is focused on the o...",0,5,"[0.854, 10.572, 0.522]","[1.464, 1.66, 4.214]",[1.21681132 1.34527799 7.84947663],0.854,1.217,10.572,1.345,0.522,7.849
76,7,,"""The paper presents a novel representation of ...",0,3,"[2.031, 21.92, 0.189]","[1.366, 1.54, 4.307]",[1.5358337 1.75166137 6.34885049],2.031,1.536,21.920,1.752,0.189,6.349
77,6,,"""This paper investigates the complexity of neu...",0,5,"[4.023, 50.659, 0.685]","[2.166, 3.052, 5.553]",[2.07553903 2.78082102 8.58274539],4.023,2.076,50.659,2.781,0.685,8.583
78,6,,"""This paper is an extension of the \u201cproto...",0,5,"[6.816, 64.86, -0.322]","[2.802, 3.496, 3.618]",[2.83244285 3.28939428 4.05086217],6.816,2.832,64.860,3.289,-0.322,4.051


In [55]:
tst_reviews

,RECOMMENDATION,REVIEW TITLE,comments,VARIANCE,CONFIDENCE,SCORES,ID,Layer 1,Layer 1 N,Layer 2,Layer 2 N,Layer 3,Layer 3 N
0,7,,"""Summary: \nThe authors present a simple varia...",0,4,"[16.587, 142.202, 0.044]",Hyp3i2xRb.json,16.587,5.481,142.202,6.059,0.044,5.699
1,4,,"""The paper entitled 'Siamese Survival Analysis...",0,5,"[1.28, 9.568, -0.34]",HkjL6MiTb.json,1.280,1.332,9.568,1.309,-0.340,3.968
2,3,,"""In this paper, the authors define a simulated...",0,3,"[11.172, 160.306, -0.672]",B1EGg7ZCb.json,11.172,4.013,160.306,6.707,-0.672,2.478
3,7,,"""Summary:\n The paper presents an unsupervised...",0,3,"[28.766, 328.435, -0.933]",HyI6s40a-.json,28.766,8.781,328.435,12.728,-0.933,1.299
4,7,,"""The paper introduces a new memory mechanism s...",0,4,"[3.718, 42.942, -0.455]",Bk9zbyZCZ.json,3.718,1.993,42.942,2.504,-0.455,3.453
...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,7,,"""The paper presents a novel representation of ...",0,3,"[2.031, 21.92, 0.189]",HkOhuyA6-.json,2.031,1.536,21.920,1.752,0.189,6.349
77,6,,"""This paper investigates the complexity of neu...",0,5,"[4.023, 50.659, 0.685]",Sy-tszZRZ.json,4.023,2.076,50.659,2.781,0.685,8.583
78,6,,"""This paper is an extension of the \u201cproto...",0,5,"[6.816, 64.86, -0.322]",HJcSzz-CZ.json,6.816,2.832,64.860,3.289,-0.322,4.051
79,4,,"""The authors tackle the problem of estimating ...",0,4,"[6.889, 85.239, -0.781]",HkjL6MiTb.json,6.889,2.852,85.239,4.019,-0.781,1.985
